In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
.builder
.appName("SparkSQLExampleApp")
.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-C8F58JN:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1618383813527)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3d55a19a


In [33]:
case class Bloggers(Id:Long, First:String, Last:String, Url:String, Published:String,
Hits:Long, Campaigns:Array[String])

defined class Bloggers


In [34]:
val bloggersDS = spark
.read
.json("C:/Users/GAME/Desktop/blogs.json")
.as[Bloggers]

bloggersDS: org.apache.spark.sql.Dataset[Bloggers] = [Campaigns: array<string>, First: string ... 5 more fields]


In [2]:
import scala.util.Random._

import scala.util.Random._


In [3]:
case class Usage(uid:Int, uname:String, usage: Int)

defined class Usage


In [4]:
val r = new scala.util.Random(42)

r: scala.util.Random = scala.util.Random@4846db1c


In [5]:
val data = for (i <- 0 to 1000)
yield (Usage(i, "user-" + r.alphanumeric.take(5).mkString(""),
r.nextInt(1000)))

data: scala.collection.immutable.IndexedSeq[Usage] = Vector(Usage(0,user-Gpi2C,525), Usage(1,user-DgXDi,502), Usage(2,user-M66yO,170), Usage(3,user-xTOn6,913), Usage(4,user-3xGSz,246), Usage(5,user-2aWRN,727), Usage(6,user-EzZY1,65), Usage(7,user-ZlZMZ,935), Usage(8,user-VjxeG,756), Usage(9,user-iqf1P,3), Usage(10,user-91S1q,794), Usage(11,user-qHNj0,501), Usage(12,user-7hb94,460), Usage(13,user-bz0WF,142), Usage(14,user-71nwy,479), Usage(15,user-7GZz1,823), Usage(16,user-1CSk6,140), Usage(17,user-WPzlL,246), Usage(18,user-VaEit,451), Usage(19,user-PSaRq,679), Usage(20,user-0Kkzu,332), Usage(21,user-UN3MG,172), Usage(22,user-KwwER,442), Usage(23,user-ZnltJ,923), Usage(24,user-IRA17,741), Usage(25,user-yNHRT,299), Usage(26,user-CJY3C,996), Usage(27,user-Yq9WW,529), Usage(28,user-RFWw1,30...


In [6]:
val dsUsage = spark.createDataset(data)
dsUsage.show(10)

+---+----------+-----+
|uid|     uname|usage|
+---+----------+-----+
|  0|user-Gpi2C|  525|
|  1|user-DgXDi|  502|
|  2|user-M66yO|  170|
|  3|user-xTOn6|  913|
|  4|user-3xGSz|  246|
|  5|user-2aWRN|  727|
|  6|user-EzZY1|   65|
|  7|user-ZlZMZ|  935|
|  8|user-VjxeG|  756|
|  9|user-iqf1P|    3|
+---+----------+-----+
only showing top 10 rows



dsUsage: org.apache.spark.sql.Dataset[Usage] = [uid: int, uname: string ... 1 more field]


In [7]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [18]:
dsUsage.where(col("usage") > 900).orderBy(desc("usage")).show(5, false)
//en el ejemplo usa filter pero da error junto a show, si se hiciese con filter se tendrian que separar ambos comandos

+---+----------+-----+
|uid|uname     |usage|
+---+----------+-----+
|605|user-NL6c4|999  |
|634|user-L0wci|999  |
|561|user-5n2xY|999  |
|113|user-nnAXr|999  |
|805|user-LX27o|996  |
+---+----------+-----+
only showing top 5 rows



In [28]:
def filterWithUsage(u: Usage) = u.usage > 900
dsUsage.filter(filterWithUsage(_)).orderBy(desc("usage")).show(5, false)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 1 in stage 7.0 failed 1 times, most recent failure: Lost task 1.0 in stage 7.0 (TID 26) (DESKTOP-C8F58JN executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @5d7aac88; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @25d95567)

In [33]:
val mapeado = dsUsage.map(u => {if (u.usage > 750) u.usage * .15 else u.usage * .50 })
.show(5, false)
def computeCostUsage(usage: Int): Double = {
if (usage > 750) usage * 0.15 else usage * 0.50
}
dsUsage.map(u => {computeCostUsage(u.usage)}).show(5, false)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 32) (DESKTOP-C8F58JN executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @5d7aac88; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @25d95567)

In [22]:
case class UsageCost(uid: Int, uname:String, usage: Int, cost: Double)

defined class UsageCost


In [23]:
def computeUserCostUsage(u: Usage): UsageCost = {
val v = if (u.usage > 750) u.usage * 0.15 else u.usage * 0.50
UsageCost(u.uid, u.uname, u.usage, v)
}

computeUserCostUsage: (u: Usage)UsageCost


In [24]:
dsUsage.map(u => {computeUserCostUsage(u)}).show(5)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 17) (DESKTOP-C8F58JN executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @224aefab; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @76f4a647)

In [26]:
case class Person(id: Integer, firstName: String, middleName: String, lastName: String,
gender: String, birthDate: String, ssn: String, salary: String)

defined class Person


In [27]:
val persons = "C:/Users/GAME/Desktop/Person.java"
val PersonDS = spark
.read
.format("java")
.option("path", persons)
.load()
.as[Person]

java.lang.ClassNotFoundException:  Failed to find data source: java. Please find packages at http://spark.apache.org/third-party-projects.html

In [ ]:
import java.util.Calendar
val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 40
personDS
.filter(x => x.birthDate.split("-")(0).toInt > earliestYear)
.filter($"salary" > 80000)
.filter(x => x.lastName.startsWith("J"))
.filter($"firstName".startsWith("D"))
.count()

personDS
.filter(year($"birthDate") > earliestYear) // Everyone above 40
.filter($"salary" > 80000) // Everyone earning more than 80K
.filter($"lastName".startsWith("J")) // Last name starts with J
.filter($"firstName".startsWith("D")) // First name starts with D
.count()